<a href="https://colab.research.google.com/github/hank199599/deep_learning_keras_log/blob/main/Chapter8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 使用LSTM產生文字資料

## 生成序列資料
訓練神經網路為先前輸入的序列資料產生接續的資料，  
該神經網路被稱為語言模型，他會去捕捉語言的淺在空間(latent space)。  
* 條件資料(conditioning data)：初始的文字字串

## 字元等級的語言模型 (charactor-level neural language model)
使用LSTM層，從文字資料庫(text corpus)中萃取N個字元的字串作為輸入，  
並訓練模型預設第N+1個字元。  
  
**模型的輸出**：所有可能的字元經softmax運算後的結果  
![pic 8-1](https://raw.githubusercontent.com/hank199599/deep_learning_keras_log/main/pictures/8-1.png)

## 取樣策略的重要性
1. greedy sampling 貪婪取樣：在所有可能字元中選擇最可能的下一個字元，但易導致重複的預測字串

# DeepDream

# 神經風格轉換

# 使用變分自編碼器 Variational Autoencoders 生成圖像

# 生成對抗神經網路簡介 Generative Adversarial Network
